In [ ]:
# Import necessary modules and classes
from langgraph.graph import StateGraph, START, END  # StateGraph manages the state machine, START and END are predefined states
from langgraph.types import Command, interrupt  # Command defines transitions and state updates, interrupt allows human intervention
from typing import TypedDict  # TypedDict is used to define structured types
from langgraph.checkpoint.memory import MemorySaver  # MemorySaver is used to save and resume state checkpoints

# Initialize a memory saver for checkpointing
memory = MemorySaver()

# Define the structure of the state used in the state machine
class State(TypedDict):
    value: str  # The state contains a single field 'value' which is a string

# Define the first node in the state machine
def node_a(state: State): 
    print("Node A")  # Log the current node
    return Command(
        goto="node_b",  # Transition to 'node_b'
        update={
            "value": state["value"] + "a"  # Append 'a' to the state's 'value' field
        }
    )

# Define the second node in the state machine
def node_b(state: State): 
    print("Node B")  # Log the current node

    # Interrupt to get human input
    human_response = interrupt("Do you want to go to C or D? Type C/D")

    print("Human Review Values: ", human_response)  # Log the human response
    
    if(human_response == "C"): 
        return Command(
            goto="node_c",  # Transition to 'node_c'
            update={
                "value": state["value"] + "b"  # Append 'b' to the state's 'value' field
            }
        ) 
    elif(human_response == "D"): 
        return Command(
            goto="node_d",  # Transition to 'node_d'
            update={
                "value": state["value"] + "b"  # Append 'b' to the state's 'value' field
            }
        )

# Define the third node in the state machine
def node_c(state: State): 
    print("Node C")  # Log the current node
    return Command(
        goto=END,  # Transition to the END state
        update={
            "value": state["value"] + "c"  # Append 'c' to the state's 'value' field
        }
    )

# Define the fourth node in the state machine
def node_d(state: State): 
    print("Node D")  # Log the current node
    return Command(
        goto=END,  # Transition to the END state
        update={
            "value": state["value"] + "d"  # Append 'd' to the state's 'value' field
        }
)

# Create a state graph to manage the workflow
graph = StateGraph(State)

# Add nodes to the state graph, each representing a state and its associated function
graph.add_node("node_a", node_a)
graph.add_node("node_b", node_b)
graph.add_node("node_c", node_c)
graph.add_node("node_d", node_d)

# Set the entry point of the state graph
graph.set_entry_point("node_a") 

# Compile the state graph into an executable application with checkpointing
app = graph.compile(checkpointer=memory)

# Define the configuration for the application
config = {"configurable": {"thread_id": "1"}}

# Define the initial state
initialState = {
    "value": ""  # Start with an empty 'value' field
}

# Invoke the application with the initial state and configuration
first_result = app.invoke(initialState, config, stream_mode="updates")
first_result


Node A
Node B


[{'node_a': {'value': 'a'}},
 {'__interrupt__': (Interrupt(value='Do you want to go to C or D? Type C/D', resumable=True, ns=['node_b:edbd727d-66fc-06b6-aa8f-17ca3c7d6ac9'], when='during'),)}]

In [ ]:
# Print the next state in the application
print(app.get_state(config).next)


('node_b',)


In [ ]:
# Resume the application from a specific state and invoke the next steps
second_result = app.invoke(Command(resume="C"), config=config, stream_mode="updates")
second_result

[]

In [ ]:
# Placeholder for additional logic or results